In [ ]:
!which pip # expect pip to be inside .../Versions/3.x/bin/pip
!echo
!which python
!echo
!which python3 # make sure python3 is a thing
!echo
!spacy # make sure spacy is a thing

/Library/Frameworks/Python.framework/Versions/3.7/bin/pip



In [ ]:
!pip install spacy

In [ ]:
!python3 -m spacy download en_core_web_sm

In [ ]:
import numpy as np
import pandas as pd
import os
import spacy

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [ ]:
DEBUG = False

try:
    # make a file called .api_key with your API_KEY from BabelNet
    with open(".api_key") as f: # https://babelnet.org/login
        BABEL_API_KEY = f.read()
        f.close()
except:
    # or just for one time use, insert it here
    BABEL_API_KEY = "<INSERT_YOUR_API_KEY>" # https://babelnet.org/login

In [ ]:
DATA_DIR = "state-of-the-state"
FILENAME = "index.csv"
SPEECH_DIR_NAME = "speeches"
TABLE_PATH = os.path.join(DATA_DIR, FILENAME)
SPEECH_DIR = os.path.join(DATA_DIR, SPEECH_DIR_NAME)
print("DATA_DIR:\t", os.listdir(DATA_DIR))
print("SPEECH_DIR[:2]:\t", os.listdir(SPEECH_DIR)[:2])
print("TABLE_PATH:\t", TABLE_PATH)
print("SPEECH_DIR:\t", SPEECH_DIR)

In [ ]:
# read the csv for info on the data
df = pd.read_csv(TABLE_PATH)
df.head()

In [ ]:
# dictionary mapping filenames to text content
speech_map = {}

# read the files
for fname in df['filename'].get_values():
    with open( os.path.join(SPEECH_DIR, fname) ) as f:
        speech_map[fname] = f.read()
        f.close()

In [ ]:
# https://spacy.io/api/span#set_extension
# extend Span object with is_stop
from spacy.tokens import Span

# if the length is 1 and the word is a stopword
stopword_checker_for_span = lambda span: (len(span) == 1 and type(span[0]).__name__ == "Token" and span[0].is_stop)
Span.set_extension("is_stop", getter=stopword_checker_for_span)

# test ._.is_stop
assert(nlp("I")[:]._.is_stop == True)
assert(nlp("Hello world")[:]._.is_stop == False)

In [ ]:
# https://spacy.io/usage/linguistic-features#noun-chunks

speech_noun_map = {}

for state in speech_map:
    doc = nlp(speech_map[state])

    nouns = []
    for chunk in doc.noun_chunks:
        if (chunk._.is_stop): # e.g. if chunk == ["it"]
            if DEBUG: 
                print("stopword?", chunk[0], chunk.text)
            continue

        if (chunk[0].text == "’s" or chunk[0].text == "'s"):
            if len(chunk) == 1:
                continue
            else:
                print("\t...found weird chunk {{", chunk, "}} ", end="")
                chunk = chunk[1:]
                print(f"...fixed... [[ {chunk} ]]")

        nouns.append(chunk.text)

    print(f"got noun chunks for {state}")
    speech_noun_map[state] = nouns

In [ ]:
print(speech_noun_map['Alabama_SOTS.txt'][:50])

In [ ]:
def query_babelnet(chunk):
    """
    given a chunk of words, do call to BabelNet API and get results
    """
    pass

def save_babelnet_results(babel_res, filename):
    """
    given BabelNet result, save to file
    """
    try:
        with open( filename, 'a' ) as f:
            f.write(babel_res)
            f.close()
    except:
        with open( filename, 'w' ) as f:
            f.write(babel_res)
            f.close()

In [ ]:
save_babelnet_results("lol\n", 'lol.txt')

In [ ]:
i=0
for state in speech_noun_map:
    nouns = speech_noun_map[state]
    i+=1
    print(i)